In [4]:
from bert_serving.client import BertClient 
bc = BertClient(check_length=False)

In [66]:
import pandas as pd
import numpy as np
from setup import get_train, get_test, get_valid
train6 = get_train(6)
test6 = get_test(6)

In [67]:
train3 = get_train(3)
test3 = get_test(3)
train2 = get_train(2)
test2 = get_test(2)

In [68]:
X_str_train = train6['statement']
y_train6 = train6['label']
X_str_test = test6['statement']
y_test6 = test6['label']

y_train3 = get_train(3)['label']
y_test3 = get_test(3)['label']
y_train2 = get_train(2)['label']
y_test2 = get_test(2)['label']

In [129]:
#bert-as-a-service recommends not pre-batching the inputs, since the program does this automatically
#however, despite reading "sent back size 10240" after the job completes, the jupyter cell still hangs
#batching prevents this issue, and lets the BERT encoding run to completion successfully
ys = []
n = len(X_str_train)
for i in range(n):
    y, _ = bc.encode(
        X_str_train.tolist()[i:(i+1)], show_tokens=True)
    ys.append(y[0])

In [130]:
#manually pad and concatenate the sentence by sentence encodings
m = max(y.shape[0] for y in ys)
X_bert_train = np.zeros((n, m, 768))
for i in range(len(ys)):
    y = ys[i]
    X_bert_train[i] = np.pad(y, ((0, m-y.shape[0]), (0,0)), 'constant')

In [131]:
#np.save('data/bert/X_bert_train', X_bert_train)

In [ ]:
#X_Bert_train = np.load('data/bert/X_bert_train.npy')
#remove pre-mean bert_train to save space

In [132]:
#average the sentence level encodings into a single 768-dimension vector
def bert_reduce_mean(X):
    return X.mean(axis=1) 

In [450]:
#X_bert_train_mean = bert_reduce_mean(X_bert_train)
#np.save('data/bert/X_bert_train_mean', X_bert_train_mean)
X_bert_train_mean = np.load('data/bert/X_bert_train_mean.npy')

In [70]:
import os
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from torch_deep_neural_classifier import TorchDeepNeuralClassifier
from torch_rnn_classifier import TorchRNNClassifier
from sklearn.metrics import classification_report

In [88]:
mod1 = TorchShallowNeuralClassifier(
    max_iter=110, hidden_dim=500)
#changing learning rate (eta) and 
#l2 regularization (l2_strength) suboptimum 
#for this model space at parameters other than default

In [89]:
y_train = y_train2
%time _ = mod1.fit(X_bert_train_mean, tuple(y_train.tolist()))

Finished epoch 110 of 110; error is 14.645215868949893

CPU times: user 3min 45s, sys: 43.2 s, total: 4min 28s
Wall time: 25.1 s


In [143]:
ys2 = []
n2 = len(X_str_test)
for i in range(n2):
    y, _ = bc.encode(
        X_str_test.tolist()[i:(i+1)], show_tokens=True)
    ys2.append(y[0])
#perform encoding for the test set

In [144]:
m2 = max(y.shape[0] for y in ys)
X_bert_test = np.zeros((n2, m2, 768))
for i in range(len(ys2)):
    y = ys2[i]
    X_bert_test[i] = np.pad(y, ((0, m2-y.shape[0]), (0,0)), 'constant')

In [145]:
#np.save('data/bert/X_bert_test', X_bert_test)
#X_bert_test = np.load('data/bert/X_bert_test.npy')

In [15]:
#X_bert_test_mean = bert_reduce_mean(X_bert_test)
#np.save('data/bert/X_bert_test_mean', X_bert_test_mean)
#X_bert_test_mean = np.load('data/bert/X_bert_test_mean.npy')
#X_bert_train_mean = np.load('data/bert/X_bert_train_mean.npy')

In [90]:
bert_train_preds = mod1.predict(X_bert_train_mean)
bert_test_preds = mod1.predict(X_bert_test_mean)

In [91]:
print(classification_report(y_train, bert_train_preds, digits=3))
print(classification_report(y_test, bert_test_preds, digits=3))

              precision    recall  f1-score   support

           0      0.420     0.464     0.441       839
           1      0.356     0.575     0.440      1995
           2      0.519     0.190     0.279      1654
           3      0.405     0.462     0.432      2114
           4      0.453     0.394     0.421      1962
           5      0.454     0.368     0.406      1676

   micro avg      0.412     0.412     0.412     10240
   macro avg      0.434     0.409     0.403     10240
weighted avg      0.432     0.412     0.403     10240

              precision    recall  f1-score   support

           0      0.248     0.326     0.282        92
           1      0.285     0.414     0.337       249
           2      0.263     0.118     0.163       212
           3      0.250     0.291     0.269       265
           4      0.258     0.224     0.240       241
           5      0.256     0.212     0.232       208

   micro avg      0.263     0.263     0.263      1267
   macro avg      0.260

In [13]:
extended_train = pd.read_csv('data/liar/liwc_train.csv')
extended_test = pd.read_csv('data/liar/liwc_test.csv')
#pre-computed LIWC-2015 corpus feature-level counts on each statement. 
#Extends the columns by ~100 features (in addition to meta-data)

In [58]:
def extend_train(subset):
    train_e = np.ndarray((extended_train.shape[0], X_bert_train_mean.shape[1]+len(subset)))
    for j in range(extended_train.shape[0]):
        train_e[j] = np.append(X_bert_train_mean[j], extended_train[subset].values[j])
    return train_e

def extend_test(subset):
    test_e = np.ndarray((extended_test.shape[0], X_bert_test_mean.shape[1]+len(subset)))
    for j in range(extended_test.shape[0]):
        test_e[j] = np.append(X_bert_test_mean[j], extended_test[subset].values[j])
    return test_e

In [59]:
all_liwc_labels = extended_train.columns[15:]

In [463]:
#extend the BERT feature vectors with the LIWC features
#X_bert_train_liwc = extend_train(all_liwc_labels)
#np.save('data/bert/X_bert_train_liwc', X_bert_train_liwc)
#X_bert_test_liwc = extend_test(all_liwc_labels)
#np.save('data/bert/X_bert_test_liwc', X_bert_test_liwc)
X_bert_test_liwc = np.load('data/bert/X_bert_test_liwc.npy')
X_bert_train_liwc = np.load('data/bert/X_bert_train_liwc.npy')

In [465]:
#extend only with the three feature functions of interest from Newman et al.
subset = ['ppron', 'negemo', 'cogproc']
#X_bert_train_liwc_rest = extend_train(subset)
#np.save('data/bert/X_bert_train_liwc_rest', X_bert_train_liwc_rest)
#X_bert_test_liwc_rest = extend_test(subset)
#np.save('data/bert/X_bert_test_liwc_rest', X_bert_test_liwc_rest)
X_bert_train_liwc_rest = np.load('data/bert/X_bert_train_liwc_rest.npy')
X_bert_test_liwc_rest = extend_test('data/bert/X_bert_test_liwc.npy')

In [60]:
#extend with a random subset of 10 feature functions
chosen = 10
subset = np.random.choice(extended_train.columns[15:], chosen)

X_bert_train_liwc_rand = extend_train(subset)
X_bert_test_liwc_rand = extend_test(subset)

In [83]:
mod2 = TorchShallowNeuralClassifier(
    max_iter=100, hidden_dim=500)

#mod3 = TorchDeepNeuralClassifier(
#    max_iter=100, hidden_dim=500)

In [95]:
#set the training set to be the extension with featureset of choice, and choose y as desired (number of labels)
train_wbl = X_bert_train_liwc_rand
test_wbl = X_bert_test_liwc_rand
y_train = y_train2
%time _ = mod2.fit(train_wbl, tuple(y_train.tolist()))

Finished epoch 100 of 100; error is 5.217316746711731

CPU times: user 3min 22s, sys: 46.4 s, total: 4min 8s
Wall time: 23.7 s


In [96]:
bert_train_preds = mod2.predict(train_wbl)
bert_test_preds = mod2.predict(test_wbl)

In [98]:
print(classification_report(y_train2, bert_train_preds, digits=3))
print(classification_report(y_test2, bert_test_preds, digits=3))

              precision    recall  f1-score   support

           0      0.891     0.433     0.583      4488
           1      0.684     0.959     0.799      5752

   micro avg      0.728     0.728     0.728     10240
   macro avg      0.788     0.696     0.691     10240
weighted avg      0.775     0.728     0.704     10240

              precision    recall  f1-score   support

           0      0.618     0.278     0.384       553
           1      0.608     0.867     0.715       714

   micro avg      0.610     0.610     0.610      1267
   macro avg      0.613     0.573     0.549      1267
weighted avg      0.613     0.610     0.570      1267

